In [6]:
%matplotlib qt
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import interactive
import numpy as np

interactive(True)

In [7]:
logfile_folder = "logfiles/"
# input_filename = "Woss_Repeater_data_20231009_1688042028_1698402213.csv"
input_filename = "Woss IT Data 2023-11-11.csv"

columns = {
    'Unix'        : 'int64',
    'Voltage_1'   : 'float32',
    'Voltage_2'   : 'float32',
    'Current'     : 'float32',
    'Power'       : 'float32',
    'Temperature' : 'float32',
    'SOC'         : 'float32',
    'Power status': 'string'
}


# Edit these values to change how the data is processed
startDate = '2021-11-01'
endDate = '2023-12-01'

moving_ave_window = 5

In [8]:
df = pd.read_csv(logfile_folder + input_filename, names=columns.keys(), sep=',', header=0, dtype=columns)
df['dateTime'] = pd.to_datetime(df['Unix'], unit='s')
columns['dateTime'] = 'string'

In [9]:
# Filter down to the requested date range
mask = (df['dateTime'] > startDate) & (df['dateTime'] <= endDate)
df = df.loc[mask]

In [10]:
movingAve_df = df.copy()

for feature_name in df.columns:
    if(feature_name == 'DateTime'):
        continue
    if(columns[feature_name] == 'string'):
        continue
    movingAve_df[feature_name] = df.rolling(window=moving_ave_window)[feature_name].mean()

# Comment out this line to not use the averaged values
df = movingAve_df

In [11]:
plt.figure()
ax = plt.gca()

df.plot(x='dateTime', y='Voltage_1', ax=ax, label='Batt Voltage')
df.plot(x='dateTime', y='Voltage_2', ax=ax, label='Solar Voltage')

plt.title(input_filename)
plt.xlabel("Date")
plt.ylabel("Voltage")

plt.tight_layout()
plt.grid()
plt.show()

In [12]:
# This is needed for the following resamples to work. It breaks things if I do it earlier though. 
df.set_index('dateTime', inplace=True)

In [13]:
# df['dateTime'] = pd.to_datetime(df['dateTime'])
# max_solar_per_day = df.groupby(df.dateTime.dt.day)['Voltage_1'].max()

# resample the battery voltage and find daily max/mins
max_values_per_day = df['Voltage_1'].resample('D').max()
min_values_per_day = df['Voltage_1'].resample('D').min()

In [18]:
# 2880 pts/day
# 1pt/30sec
solar_above_12V = df[df['Voltage_2'] > 10]
solar_minutes_above_12V_per_day = solar_above_12V.resample('D').count() / 2
# print(solar_minutes_above_12V_per_day)

solar_minutes_above_12V_per_day['Voltage_2'].plot(label='Minutes of solar power per day')
plt.xlabel('Date')
plt.ylabel('Number of minutes of solar power')
plt.title('Number of minutes of solar power (voltage above 12V) per day')
plt.grid(True)
plt.show()

In [15]:
max_values_per_day.plot(marker='o', linestyle='-', label='maximum voltage', title='Max/Min Battery Voltage per Day')
min_values_per_day.plot(marker='o', linestyle='-', label='minimum voltage')
plt.xlabel('Date')
plt.ylabel('Battery Voltage')
plt.grid(True)
plt.legend()
plt.show()